In [ ]:
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE RankNTypes #-}
import Control.Monad
import Control.Monad.Primitive

import Data.Function
import Data.Ord
import qualified Data.Vector.Generic as G
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants (m_epsilon)
import Numeric.MathFunctions.Comparison

import Statistics.Distribution
import Statistics.Distribution.Beta
import Statistics.Distribution.ChiSquared
import Statistics.Distribution.Gamma
import Statistics.Distribution.StudentT
import Statistics.Distribution.FDistribution
import Statistics.Distribution.Laplace

import Statistics.Test.WilcoxonT
import Statistics.Test.Internal
import Statistics.Function

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy

:l NB/Plot

In [ ]:
d = laplace (-1.8615772269443358)  0.7052041610376716

toRenderable $ plotFunctions [density d] (-10,10)
toRenderable $ plotFunctions [ cumulative d
                             , complCumulative d
                             ] (-10,10)
toRenderable $ plotFunctions [ quantile d
                             , complQuantile d
                             ] (0.01,0.99)

In [ ]:
testData :: [([Double], [Double], (Int,Double, Double))]
testData = [ ([1..10], [1..10], (0, 0, 0     ))
               , ([1..5],  [6..10], (0, 0, 5*(-3)))
               -- Worked example from the Internet:
               , ( [125,115,130,140,140,115,140,125,140,135]
                 , [110,122,125,120,140,124,123,137,135,145]
                 , ( 0
                   , sum $ filter (> 0) [7,-3,1.5,9,0,-4,8,-6,1.5,-5]
                   , sum $ filter (< 0) [7,-3,1.5,9,0,-4,8,-6,1.5,-5]
                   )
                 )
               -- Worked examples from books/papers:
               , ( [2.4,1.9,2.3,1.9,2.4,2.5]
                 , [2.0,2.1,2.0,2.0,1.8,2.0]
                 , (0, 18, -3)
                 )
               , ( [130,170,125,170,130,130,145,160]
                 , [120,163,120,135,143,136,144,120]
                 , (0, 27, -9)
                 )
               , ( [540,580,600,680,430,740,600,690,605,520]
                 , [760,710,1105,880,500,990,1050,640,595,520]
                 , (0, 3, -42)
                 )
               ]

let (a,b,c) = testData !! 1
 in wilcoxonMatchedPairSignedRank (U.zip (U.fromList a) (U.fromList b))

In [ ]:
5 :: Double

In [ ]:
wilcoxonMatchedPairSignedRank' ab
  = (nRed, ranks1, ranks2)
  where
    -- Positive and negative ranks
    (ranks1, ranks2) = splitByTags
                     $ U.zip tags (rank ((==) `on` abs) diffs)
    -- Sorted list of differences
    diffsSorted = sortBy (comparing abs)    -- Sort the differences by absolute difference
                $ U.filter  (/= 0)          -- Remove equal elements
                $ U.map (uncurry (-)) ab    -- Work out differences
    nRed = U.length diffsSorted
    -- Sign tags and differences
    (tags,diffs) = U.unzip
                 $ U.map (\x -> (x>0 , x))   -- Attach tags to distribution elements
                 $ diffsSorted
let (a,b,c) = testData !! 1
 in wilcoxonMatchedPairSignedRank' (U.zip (U.fromList a) (U.fromList b))

In [ ]:
invIncompleteBeta 1 1 1e-250

In [ ]:
incompleteBeta 1 1 1e-250